In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# About this notebook
I uses this notebook to learn NLP myself. Here, I does copy a lot of other notebooks (with citations :D) to get fast understanding in the field.

I adds explanation and re-arrange stuff in order that beginners like me can easily follow.

If you find it useful for yourself, please upvote for me and the orgiginal notebook also.

Thank you so much for reading!

The contents include
1. Understanding the data set and requirements: Discuss what we have been given and what we need to do.
2. Data Preprocessing: Prepare data that are ready to feed to any NN models
3. NN models: Discuss what NN models, machine learning techniques we are going to explore
4. Trainings: Training our models and discuss the results
5. Finish up: Transfer the results to submission.csv

# 1. Understanding the Data Set and Requirements

Jigsaw Multilingual Toxic Comment Classification: Use TPUs to identify toxicity comments across multiple languages.

<span style="color:red"> **Simplest explanation**</span>: Given a comment in a language => classify that comment is toxic or non-toxic.

The data set includes these files and the files have 4 columns

<span style="color:red"> **Files**</span>
* **jigsaw-toxic-comment-train.csv** - data from the 1st competition. https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

The dataset is made up of English comments from Wikipedia’s talk page edits.

* **jigsaw-unintended-bias-train.csv** - data from the 2nf competition. https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification

This is an expanded version of the Civil Comments dataset with a range of additional labels.


* **validation.csv** - comments from Wikipedia talk pages in different non-English languages.
* **test.csv** - comments from Wikipedia talk pages in different non-English languages.


* **jigsaw-toxic-comment-train-processed-seqlen128.csv** - training data preprocessed for **BERT**
* **jigsaw-unintended-bias-train-processed-seqlen128.csv** - training data preprocessed for **BERT**
* **validation-processed-seqlen128.csv** - validation data preprocessed for **BERT**
* **test-processed-seqlen128.csv** - test data preprocessed for **BERT**


* **sample_submission.csv** - a sample submission file in the correct format


<span style="color:red"> **Columns**</span>
* **id** - identifier within each file.
* **comment_text** - the text of the comment to be classified.
* **lang** - the language of the comment.
* **toxic** - whether or not the comment is classified as toxic (1 vs 0). (Does not exist in test.csv.)

# Explanation of data set files

**jigsaw-toxic-comment-train.csv** have **223549** rows of **comment_text** is classified into 6 levels of toxicity

In [ ]:
DIR_INPUT= '/kaggle/input/jigsaw-multilingual-toxic-comment-classification'
train_df1 = pd.read_csv(DIR_INPUT + '/jigsaw-toxic-comment-train.csv')
train_df1.head()

In [ ]:
print(train_df1.shape)

**jigsaw-unintended-bias-train.csv** have **1902194** rows of **comment_text** and 43 columns of information about comments

In [ ]:
train_df2 = pd.read_csv(DIR_INPUT + '/jigsaw-unintended-bias-train.csv')
train_df2.head()

In [ ]:
print(train_df2.shape)
print(train_df2.columns)

**validation.csv** have **8000** rows of **comment_text** and 2 columns of languages and toxic categories

In [ ]:
validation_df = pd.read_csv(DIR_INPUT + '/validation.csv')
validation_df.head()

In [ ]:
print(validation_df.shape)

# Requirements

**test.csv** have **63812** rows of **comment_text** in multi-languages and 1 columns of language types.

<span style="color:red"> **We need to classify these comments as toxic or non-toxic, and then put our prediction to the sample_submission file**</span>

<span style="color:red"> **We can use all above files as our trainning/validation data set**</span>

In [ ]:
test_df = pd.read_csv(DIR_INPUT + '/test.csv')
test_df.head()

In [ ]:
print(test_df.shape)

**sample_submission.csv** have **63812** rows of **id** and **toxic** classification.

<span style="color:red"> **We need to replace the value of the toxic columns with our prediction with respect to the id in test.csv files..**</span>

In [ ]:
sample_submission= pd.read_csv(DIR_INPUT + '/sample_submission.csv')
sample_submission.head()

In [ ]:
print(sample_submission.shape)

# 2. Data Preprocessing
Here, we will prepare our data to be ready for feeding any NN models.